In [1]:
!pip install ultralytics --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
from google.colab import files
import os
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [27]:
uploaded = files.upload()
input_path = list(uploaded.keys())[0]


Saving car1.mp4 to car1 (1).mp4


In [4]:
model = YOLO('yolov5s.pt')


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 81.7MB/s]


In [ ]:
import cv2
import time
import numpy as np

# Initialize Kalman Filter
kalman = cv2.KalmanFilter(4, 2)  # 4 states (x, y, dx, dy), 2 measurements (x, y)

# Define transition matrix (A)
kalman.transitionMatrix = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype=np.float32)

# Measurement matrix (H)
kalman.measurementMatrix = np.eye(2, 4, dtype=np.float32)

# Process and measurement noise covariances
kalman.processNoiseCov = 1e-2 * np.eye(4, dtype=np.float32)
kalman.measurementNoiseCov = 1e-1 * np.eye(2, dtype=np.float32)
kalman.errorCovPost = 1. * np.eye(4, dtype=np.float32)

# Initial state (will be set on first detection)
initialized = False

# Video setup
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = 'tracked_kalman.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_id = 0
class_names = model.names

while True:
    start_time = time.time()

    success, frame = cap.read()
    if not success:
        break

    results = model.track(
        source=frame,
        persist=True,
        verbose=False,
        imgsz=960,
        conf=0.25,
        iou=0.45
    )

    filtered_boxes = [
        box for box in results[0].boxes
        if int(box.cls.item()) in [0, 2]
    ]
    results[0].boxes = filtered_boxes

    annotated_frame = frame.copy()

    if filtered_boxes:
        # Use the first box for demo purposes
        box = filtered_boxes[0]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2

        measurement = np.array([[np.float32(cx)], [np.float32(cy)]])

        if not initialized:
            kalman.statePost = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
            initialized = True

        # Kalman prediction and correction
        predicted = kalman.predict()
        kalman.correct(measurement)

        # Get smoothed coordinates
        px, py = int(predicted[0]), int(predicted[1])

        # Draw red circle at predicted center
        cv2.circle(annotated_frame, (px, py), 8, (0, 0, 255), -1)

        # Draw label and bounding box
        cls_id = int(box.cls.item())
        label = class_names[cls_id]
        text = f"{label}"
        cv2.putText(annotated_frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # FPS overlay
    end_time = time.time()
    processing_fps = 1 / (end_time - start_time + 1e-6)
    cv2.putText(annotated_frame, f"FPS: {processing_fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    out.write(annotated_frame)
    frame_id += 1

cap.release()
out.release()


/tmp/ipython-input-28-379348551.py:81: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  px, py = int(predicted[0]), int(predicted[1])


In [26]:
from google.colab import files
files.download('tracked_kalman.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>